In [50]:
from pyqubo import UserDefinedExpress, Array, Xor, And, Or, AndConst, OrConst, XorConst, Binary, Spin, Integer
import dimod
import neal

In [4]:
from IPython.display import IFrame

## Half and Full Adder

<img src="./images/half_adder.png" alt="half-adder" align="left" style="width: 300px;"/>

In [51]:
class HalfAdder(UserDefinedExpress):
    def __init__(self, a, b, s, c, label):
        xor_const = XorConst(a, b, s, label=label+'_xor')  
        and_const = AndConst(a, b, c, label=label+'_and')
        H = xor_const + and_const
        super().__init__(H)

<img src="./images/full_adder.png" alt="full-adder" align="left" style="width: 600px;"/>

In [58]:
class FullAdder(UserDefinedExpress):
    def __init__(self, a, b, c_in, s, c_out, label):
        s1 = Binary(label+'_s1')
        c1 = Binary(label+'_c1')
        c2 = Binary(label+'_c2')
        h1 = HalfAdder(a, b, s1, c1, label+'_FA_HA1')
        h2 = HalfAdder(s1, c_in, s, c2, label+'_FA_HA2')
        or_const = OrConst(c1, c2, c_out, label+'_FA_or')
        H = h1 + h2 + or_const
        super().__init__(H)

## Two-Bit Multiplier

In [59]:
class TwoBitMultiplier(UserDefinedExpress):
    
    def __init__(self, a, b, p, label):
        a0, a1 = a 
        b0, b1 = b
        p0, p1, p2, p3 = p
        
        b0a0 = Binary(label+'_b0a0') 
        b0a1 = Binary(label+'_b0a1')
        b1a0 = Binary(label+'_b1a0')
        b1a1 = p3 # Binary(f"{label}_b1a1")
        
        and_1 = AndConst(b1, a1, b1a1, label+'_and1') 
        and_2 = AndConst(b1, a0, b1a0, label+'_and2')
        and_3 = AndConst(b0, a1, b0a1, label+'_and3')
        and_4 = AndConst(a0, b0, b0a0, label+'_and4')  
        
        ha1_c = Binary("ha1_c")  
        
        ha1 = HalfAdder(b1a0, b0a1, p2, ha1_c, label+'_ha1')  # ha1_s
        ha2 = HalfAdder(b0a0, ha1_c, p1, p0, label+'_ha2')  # ha2_s
        
        H = and_1 + and_2 + and_3 + and_4 + ha1 + ha2
        super().__init__(H)

In [62]:
def find_two_bit_multiplier_solutions(num_sol=5):
    x = Array.create('x', shape=(2), vartype='BINARY')
    y = Array.create('y', shape=(2), vartype='BINARY')
    p = Array.create('p', shape=(4), vartype='BINARY')
    H = TwoBitMultiplier(x, y, p, "mult")
    model = H.compile()
    qubo, offset = model.to_qubo()
    all_solutions = dimod.ExactSolver().sample_qubo(qubo)
    best_n_solutions = min(model.decode_sampleset(all_solutions), key=lambda x: x.energy)
    return best_n_solutions

In [63]:
two_bit_multiplier_solutions = find_two_bit_multiplier_solutions(num_sol=5)

In [64]:
two_bit_multiplier_solutions.sample

{'aux_mult_ha2_xor': 0,
 'mult_b0a1': 0,
 'ha1_c': 0,
 'p[1]': 0,
 'mult_b0a0': 0,
 'mult_b1a0': 0,
 'aux_mult_ha1_xor': 0,
 'p[0]': 0,
 'p[2]': 0,
 'p[3]': 0,
 'y[1]': 0,
 'x[0]': 0,
 'y[0]': 0,
 'x[1]': 0}

## Three-Bit Multiplier

<img src="./images/multiplier.png" alt="multiplier" align="left" style="width: 700px;"/>

In [68]:
class ThreeBitMultiplier(UserDefinedExpress):
    
    def __init__(self, a, b, p, label): 
        a0, a1, a2 = a 
        b0, b1, b2 = b
        p0, p1, p2, p3, p4, p5 = p
        
        b0a0 = Binary(label+"_b0a0") 
        b0a1 = Binary(label+"_b0a1")
        b1a0 = Binary(label+"_b1a0")
        b1a1 = Binary(label+"_b1a1")
        b0a2 = Binary(label+"_b0a2")
        b2a0 = Binary(label+"_b2a0")
        b1a2 = Binary(label+"_b1a2")
        b2a1 = Binary(label+"_b2a1")
        b2a2 = p5 # Binary(label+"_b2a2")
        
        and_9 = AndConst(a0, b0, b0a0, label+"_and9")  # b0a0
        and_8 = AndConst(b0, a1, b0a1, label+"_and8")
        and_7 = AndConst(b1, a0, b1a0, label+"_and7")
        and_6 = AndConst(b1, a1, b1a1, label+"_and6")
        and_5 = AndConst(b0, a2, b0a2, label+"_and5")
        and_4 = AndConst(b2, a0, b2a0, label+"_and4")
        and_3 = AndConst(b1, a2, b1a2, label+"_and3")
        and_2 = AndConst(b2, a1, b2a1, label+"_and2")
        and_1 = AndConst(b2, a2, b2a2, label+"_and1")
         
        ha1_c = Binary("ha1_c")
        fa1_c = Binary("fa1_c")
        fa1_s = Binary("fa1_s")
        ha2_c = Binary("ha2_c")
        fa2_c = Binary("fa2_c")
        fa2_s = Binary("fa2_s")
        ha3_c = Binary("ha3_c")
        
        ha1 = HalfAdder(b2a1, b1a2, p4, ha1_c, label+"_ha1")  # ha1_s
        fa1 = FullAdder(b1a1, b2a0, ha1_c, fa1_s, fa1_c, label+"_fa1")
        
        ha2 = HalfAdder(b0a2, fa1_s, p3, ha2_c, label+"_ha2")  # ha2_s
        fa2 = FullAdder(b1a0, fa1_c, ha2_c, fa2_s, fa2_c, label+"_fa2")
        
        ha3 = HalfAdder(b0a1, fa2_s, p2, ha3_c, label+"_ha3")  # ha3_s
        fa3 = FullAdder(b0a0, fa2_c, ha3_c, p1, p0, label+"_fa3")  # fa3_c, fa3_s
        
        H = and_1 + and_2 + and_3 + and_4 + and_5 + and_6 + and_7 + and_8 + and_9 \
            + ha1 + ha2 + ha3 + fa1 + fa2 + fa3
        super().__init__(H)

In [75]:
def decode_bits(bits):
    """Define the function to decode the bit array into an integer"""
    return sum(x*2**i for i, x in enumerate(bits[::-1]))

def find_three_bit_multiplier_factors(product=[0, 0, 0, 0, 0, 0]):
    """
    Find integers a, b such that a×b=p
    
    Args:
        product: binary array to represent the integer p
    
    Returns:
        tuple of binary arrays of a and b
    """
    x = Array.create('a', shape=(3), vartype='BINARY')
    y = Array.create('b', shape=(3), vartype='BINARY')
    H = ThreeBitMultiplier(x, y, product, "mult")
    model = H.compile()
    qubo, offset = model.to_qubo()
    sampler = neal.SimulatedAnnealingSampler()
    sol = sampler.sample_qubo(qubo, num_reads=100, beta_range=(100.,10000.))
    decoded_threebit = min(model.decode_sampleset(sol), key=lambda x: x.energy)
    a_bits = [decoded_threebit.sample[f"a[{i}]"] for i in range(3)]
    b_bits = [decoded_threebit.sample[f"b[{i}]"] for i in range(3)]
    return a_bits, b_bits

In [76]:
p_bits = [0, 0, 1, 0, 1, 0]
a_bits, b_bits = find_three_bit_multiplier_factors(p_bits)
print("Bit array of a is", a_bits)
print("Bit array of b is", b_bits)

Bit array of a is [0, 1, 0]
Bit array of b is [1, 0, 1]


In [80]:
# we show the decoded values of a, b, and p
p = decode_bits(p_bits)
a = decode_bits(a_bits)
b = decode_bits(b_bits)
print(f"p={p}")
print(f"a={a}")
print(f"b={b}")

p=10
a=2
b=5


In [82]:
# we can confirm that a*b=p
assert a*b==p